# Code de simulation d'images 

## 0.Introduction et explications générales 

Ce code à pour objectif de créer une image simulée à partir d'un fichier .dat fourni par le code AFM de Jérémie Vaubaillon. Dans notre cas il s'appelle "STEPS_int.dat0" et sera lu dans la cellule suivante. 

Ce code suit la feuille de route fournie par Jérémie lors du projet meteorix 2023, et en suit donc les 4 étapes. 

In [1]:
"""Importation de modules et définition de constantes""" 
import numpy as np 
from astropy.io import fits 
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib qt5

h = 6.63e-34 #J/s
c = 3.00e8 #m/s
lambda_moyen = 600e-9 # m

"""Lecture du fichier et des données utiles""" 
file = np.loadtxt("STEPS_int.dat0").T
vitesse_meteoroide = 66000       #m/s expliquée dans la feuille de route
Xm = file[8]                     #m
Ym = file[9]                     #m 
dm_dt = file[11]                 #kg/s
altitude_meteore = file[5] * 1e3 #m
t = file[1]                      #s 
nb_lignes = len(t)

La cellule suivante regroupe la totalité des paramètres de la caméra ou d'ailleurs utiles au code, il pourront être modifiés si ils sont jugés impertinents ou imprecis. Ils sont sourcés afin de garder une trace des informations. 

In [2]:
"""Paramètres de la caméra"""  

altitude_meteorix = 500e3 #m 
#Source : Article meteorix

taille_pixels = 5*5 * 1e-12 #m²
#Source : Rapport 2022 (p12)

distance_focale = 8 * 1e-3 #m 
#Source : Rapport 2022 (p11)

diametre_camera = 11.2 * 1e-3 #m 
#Source : Rapport 2022 

profondeur_camera = 59 * 1e-3 #m 
# Source : Rapport 2022 (Annexe)

angle_camera = 40 #° 
#source : article meteorix (100° pour la petite caméra)

largeur_detecteur = 640  # pixels 
hauteur_detecteur = 480  # pixels
#Source : observations à l'aide de la caméra 2023 

## 1. Physique des météores

Cette partie à pour objectif de déterminer l’énergie lumineuse émise par un météoroïde lorsque celui-ci s'ablate en entrant dans l'atmosphère. 

In [3]:
# Etape 1 

tau = 0.05 # sans unité 
#Cette valeur a été fournie avec la feuille de route

#Calcul de l’énergie lumineuse émise : 
I_emis = - tau * vitesse_meteoroide*vitesse_meteoroide * dm_dt # W 
#Source de la formule : TP nanosatellite 2018 par Nicolas Rambaux 

## 2. Signal détecté

Cette partie vise à déterminer le signal détecté par la caméra à partir de différents paramètres que nous obtiendrons via des sources bibliographiques. 


Il faut corriger : 
- La distance entre le phénomène et le détecteur
- L’efficacité quantique du détecteur : Q=70%
- l’étalage de la lumière sur le détecteurs, via la PSF (point spread function), la taille des pixels et la longueur focale de la caméra.

Approximations :     
- On considère le météoride et la caméra alignés avec le centre de la Terre -> le météoride est pile au centre du champs de vision de la caméra

In [4]:
distance_meteore_camera = np.sqrt((altitude_meteorix - altitude_meteore)**2 + (Xm)**2)
#Calculée par nous, on peux utiliser le théorème de pythagore pour le démontrer 

efficacite_quantique = 0.7 
#Fournie par la feuille de route; il s'agit d'un pourcentage 

intensite_recue = I_emis / (distance_meteore_camera)**2
#Source : Discussion du 15/02 avec Jérémie et Nicolas. 

a = 0 
delta_temps = np.array([])
for i in t : 
    delta_temps = np.append(delta_temps, i - a)
    a = i 
#boucle qui calcule l'écart de durée entre 2 lignes du fichier en secondes, stockées dans "delta_temps"

nombre_photons = intensite_recue * taille_pixels * delta_temps * lambda_moyen /  (h * c)
#Nombre de photons captés chaque intervalle de temps (= énergie recue/energie d'un photon)

surface_airy = np.pi * (profondeur_camera * np.sin(1.22 * lambda_moyen / diametre_camera ) )**2 
#Calcul de la diffraction du a la camera en mètre (revoir si 1.22 lambda / D est une bonne approximation)

proportion_lumiere_par_pixel = taille_pixels / surface_airy
#Calcul de l'etalage de la lumière du a la diffraction 

signal_detecte = proportion_lumiere_par_pixel * efficacite_quantique * nombre_photons 
# en unité adimensionnee "ADU"

for i in range(len(signal_detecte)) : 
    if signal_detecte[i] > 255 : 
        signal_detecte[i] = 255.
    if signal_detecte[i] < 0 : 
        signal_detecte[i] = 0
#Cette boucle à pour objectif de regler les pixels saturés a 255 (dans notre cas ils le sont tous)

## 3. Image artificielle

Dans cette partie l'objectif est de créer des images de ce météoroïde en fonction du temps

In [6]:
#Dans le cas de "STEPS_int.dat0" on crée 14 images noires de 480x640
images = np.resize(np.zeros(nb_lignes*hauteur_detecteur*largeur_detecteur),(nb_lignes,hauteur_detecteur,largeur_detecteur))

#On calcule la largeur de l'image à l'endroit du meteoroide 
largueur_image = np.abs(2 * np.tan(angle_camera*np.pi/180) * distance_meteore_camera) #m 

#On détermine le pixel à illuminer grâce à la position du météoroïde 
x_pixel = largeur_detecteur/2 + (largeur_detecteur * Xm / largueur_image )
y_pixel = (hauteur_detecteur * Ym / largueur_image ) + hauteur_detecteur/2


for i in range(nb_lignes) : 
    """Boucle qui crée les images et les place dans le tableau images"""
    image_a_ajt = np.resize(np.zeros(hauteur_detecteur*largeur_detecteur),(hauteur_detecteur,largeur_detecteur))
    
    #On modifie le pixel qui correspond par une valeur (cette valeur sera calculée à partir de I plus tard)
    image_a_ajt[int(y_pixel[i])][int(x_pixel[i])] = signal_detecte[i] #Valeur pour un objet de magnitude 1.85
    
    #On calcule la valeur des pixels adjacents à partir de la surface de la tache d'Airy
    val_pixel_adj = (1 - proportion_lumiere_par_pixel) * signal_detecte[i]/8 

    #On modifie les 8 pixels adjascents par la valeur calculée 
    image_a_ajt[int(y_pixel[i]) +1][int(x_pixel[i])] = val_pixel_adj
    image_a_ajt[int(y_pixel[i])][int(x_pixel[i]) +1] = val_pixel_adj
    image_a_ajt[int(y_pixel[i] -1)][int(x_pixel[i])] = val_pixel_adj
    image_a_ajt[int(y_pixel[i])][int(x_pixel[i]) -1] = val_pixel_adj
    image_a_ajt[int(y_pixel[i]) +1][int(x_pixel[i]) +1] = val_pixel_adj
    image_a_ajt[int(y_pixel[i]) -1][int(x_pixel[i]) -1] = val_pixel_adj
    image_a_ajt[int(y_pixel[i]) +1][int(x_pixel[i]) -1] = val_pixel_adj
    image_a_ajt[int(y_pixel[i]) -1][int(x_pixel[i]) +1] = val_pixel_adj
    
    #On ajoute l'image formée dans le tableau 
    images[i] = image_a_ajt


#Paramètres de la figure
fig=plt.figure(figsize=(10,10))
ax=fig.add_subplot(111)
fig.show()
ax.set_xlim(0,largeur_detecteur)
ax.set_ylim(0,hauteur_detecteur)

#Affiche les images formées sous forme d'une animation 
for i in range(nb_lignes):
    ax.imshow(images[i], cmap='gray')
    plt.pause(0.1)
    fig.canvas.draw()
plt.pause(.1)